In [3]:
"""返回函数

高阶函数除了可以把函数作为参数外，还可以把函数作为结果值返回
"""

# 定义一个可变参数的求和函数
def calc_sum(*args):
    s = 0
    for n in args:
        s = s + n
    return s

print(calc_sum(1, 2, 3))


# 不需要立即求和，根据需要再进行计算
# 即不返回求和的结果，而是返回求和的函数
def lazy_sum(*args):
    def calc_sum():
        s = 0
        for n in args:
            s = s + n
        return s
    return calc_sum

f = lazy_sum(1, 2, 3)
print(f)
print(f())  # 调用时才真正求和

6
<function lazy_sum.<locals>.calc_sum at 0x0000017D55E0A158>
6


In [5]:
"""闭包函数"""

# 使用闭包时：返回函数不要引用任何循环变量，或者后续会发生变化的变量
def count():
    fs = []
    for i in range(1, 4):
        def f():
            return i * i
        fs.append(f)
    return fs

f1, f2, f3 = count()
print('f1:', f1())
print('f2:', f2())
print('f3:', f3())


# 对以上函数进行改进
# 方法是再创建一个函数，用该函数的参数绑定循环变量的当前值
# 无论该循坏变量后续如何变化，已绑定到函数参数的值不会变
def count2():
    def f(i):
        def g():
            return i * i
        return g
    
    fs = []
    for n in range(1, 4):
        fs.append(f(n))  # f(n)立刻被执行，因此n的当前值被传入f()
    return fs

f1, f2, f3 = count2()
print('f1:', f1())
print('f2:', f2())
print('f3:', f3())

f1: 9
f2: 9
f3: 9
f1: 1
f2: 4
f3: 9


In [1]:
"""lambda表达式

定义匿名函数和内联函数
只能定义单个表达式，该表达式运算的结果就是函数的返回值
"""

# 使用lambda表达式定义匿名函数，并赋值给一个变量
add = lambda x, y: x + y

# 测试
print(add(1, 2))

3


In [2]:
"""lambda的应用场景

数据排序sorted、reduce等
"""

# 对列表中的姓名按last name进行排序
names = ['Kobe Brynent', 'Jemmy Lin', 'Curry Steven', 'Keven Durant', 'Lebron James', 'Michale Jordan']

sorted(names, key = lambda name: name.split()[-1].lower())

['Kobe Brynent',
 'Keven Durant',
 'Lebron James',
 'Michale Jordan',
 'Jemmy Lin',
 'Curry Steven']

In [3]:
"""使用匿名函数捕获变量值

普通函数默认参数的值是在定义时就绑定的
匿名函数参数的值是自由变量，是在执行时才绑定的

要让匿名函数的参数在定义时就捕获到值，可将该参数定义为默认参数
"""

# 测试
x = 10
add1 = lambda y: x + y

x = 20
add2 = lambda y: x + y

print(add1(10))
print(add2(10))

# 测试
a = 100
add3 = lambda b, a=a: a + b

a = 200
add4 = lambda b, a=a: a + b

print(add3(100))
print(add4(100))

30
30
200
300


In [4]:
"""减少可调用对象callable的参数个数

偏函数partial()固定某些参数，并返回一个新的callable对象
简化函数调用
"""

from functools import partial


def test(a, b, c, d):
    print(a, b, c, d)
    
    
# 测试
t1 = partial(test, 1)  # a = 1
t1(2, 3, 4)

t2 = partial(test, d=4)  # d = 4
t2(1, 2, 3)

t3 = partial(test, 1, d=4)  # a = 1, d = 4
t3(2, 3)

1 2 3 4
1 2 3 4
1 2 3 4


In [5]:
"""偏函数的使用举例"""

from functools import partial
import math


# 用列表保存多个点的坐标，每一个点为一个元祖
points = [(3, 2), (1, 2), (2, 4), (2, 2), (1, 1), (3, 4)]


# 定义函数计算两点之间的距离
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)


# 以与基点(0, 0)之间的距离进行排序
pt = (0, 0)
points.sort(key = partial(distance, pt))
points

[(1, 1), (1, 2), (2, 2), (3, 2), (2, 4), (3, 4)]

In [6]:
"""回调函数

回调函数常用在事件处理、等待后台任务完成后的回调等
"""


# 定义并调用回调函数
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)
    
    
def add(x, y):
    return x + y


def print_r(result):
    print('Result: ', result)
    
    
# 测试
apply_async(add, (1, 2), callback=print_r)
apply_async(add, ('hello', 'world'), callback=print_r)

Result:  3
Result:  helloworld


In [8]:
"""使用闭包来捕获状态值"""


def make_handler():
    seq = 0
    
    def handler(result):
        nonlocal seq  # 声明变量将在闭包函数中被修改
        seq += 1
        print('[{}] Result: {}'.format(seq, result))
        
    return handler


# 测试
handler = make_handler()
apply_async(add, (1, 2), callback = handler)
apply_async(add, ('hello', 'world'), callback = handler)

[1] Result: 3
[2] Result: helloworld


In [7]:
"""让回调函数带额外的状态值

使用绑定方法来代替简单的回调函数
"""


class ResultHandler:
    def __init__(self):
        self.seq = 0
        
    def handler(self, result):
        self.seq += 1
        print('[{}] Result: {}'.format(self.seq, result))
        
        
# 测试
rh = ResultHandler()
apply_async(add, (1, 2), callback = rh.handler)
apply_async(add, ('hello', 'world'), callback = rh.handler)

[1] Result: 3
[2] Result: helloworld


In [9]:
"""访问闭包中的变量

通常来讲，闭包中的变量对于外部是隐藏的
通过编写访问函数，并将其作为属性绑定到闭包函数上来实现访问闭包内部变量的目的
"""


def func():
    n = 0
    
    def print_n():
        print('n = ', n)
        
    def get_n():
        return n
    
    def set_n(val):
        nonlocal n
        n = val
        
    # 把访问函数绑定为闭包函数的属性
    print_n.get_n = get_n
    print_n.set_n = set_n
    
    # 不是所有的内部函数都是闭包函数，只有被返回的那个才是
    return print_n


# 测试
f = func()
f()

f.set_n(1)
f()

f.set_n(2)
f.get_n()

n =  0
n =  1


2

In [10]:
"""递归函数

在函数内部调用自身的函数
递归函数的优点是定义简单，逻辑清晰

用递归函数需要注意防止栈溢出
函数调用是通过栈（stack）这种数据结构实现的，每当进入一个函数调用，栈就会加一层栈帧，每当函数返回，栈就会减一层栈帧
由于栈的大小不是无限的，所以，递归调用的次数过多，会导致栈溢出

解决递归调用栈溢出的方法是通过尾递归优化
尾递归事实上和循环是等价的
"""

# 求阶乘函数
def fact(n):
    if n == 1:
        return 1
    return fact(n-1) * n

print('5! =', fact(5))

5! = 120


In [7]:
"""装饰器

函数也是一个对象，而且函数对象可以被赋值给变量，所以通过变量也能调用该函数
可以通过函数对象的__name__属性获取函数的名字
"""

def hello():
    print('hello world')
    
h = hello  # 把函数赋值给变量

hello()
h()

print(hello.__name__)
print(h.__name__)

hello world
hello world
hello
hello


In [15]:
"""装饰器

在代码运行期间动态增加功能的方式，称之为装饰器Decorator
即要增强原函数的功能，但又不修改原函数的定义
装饰器的本质是返回原函数的高阶函数
"""
import functools


# 定义一个打印日志的装饰器函数
# 它接受一个函数作为参数，并返回一个函数
def log(func):
    @functools.wraps(func)  # 把原函数的__name__等属性复制到warpper()函数中
    def wrapper(*args, **kw):
        print('Call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper


# 使用装饰器
# 相当于执行了：hello = log(hello)
@log
def hello():
    print('hello world')
    
hello()

# print(hello.__name__)

Call hello():
hello world


In [18]:
"""装饰器

如果装饰器本身需要传入参数，那就需要多加一层
"""
import functools

# 自定义log文本
def log(txt):
    def decorator(func):
        @functools.wraps(func)  # 把原函数的__name__等属性复制到warpper()函数中
        def wrapper(*args, **kw):
            print('%s %s():' % (txt, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator


# 使用装饰器
# # 相当于执行了：hello = log('Execute')(now)
@log('Execute')
def hello():
    print('hello world')
    
hello()

# print(hello.__name__)

Execute hello():
hello world
